# Reflection Pattern with LangGraph and ChatAmazonNova

This notebook demonstrates an agent that critiques and improves its own outputs through iterative reflection.

## Process Flow
1. **Generate**: Create initial content
2. **Reflect**: Critique the output
3. **Generate**: Revise based on critique
4. **Repeat**: Until approved or max iterations reached

## Key Concepts
- Self-improvement loops
- Quality iteration
- Conditional termination

In [ ]:
%env NOVA_API_KEY=<YOUR-API-KEY>
%env NOVA_BASE_URL=https://api.nova.amazon.com/v1/

## Setup and Imports

In [12]:
from typing import TypedDict, List

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.graph import StateGraph, START, END

from langchain_amazon_nova import ChatAmazonNova

## Define State

In [13]:
class ReflectionState(TypedDict):
    """State for the reflection loop."""
    messages: List
    iterations: int
    max_iterations: int
    initial_query: str

## Create Generator and Reflector

The generator creates content, the reflector critiques it.

In [14]:
def create_generator(llm):
    """Create the content generator."""
    
    system_message = SystemMessage(
        content="""You are a helpful assistant that generates content based on user requests.
        If you receive critique or feedback, revise your previous response to address the concerns."""
    )
    
    def generate(state: ReflectionState) -> dict:
        messages = [system_message] + state["messages"]
        response = llm.invoke(messages)
        
        return {
            "messages": state["messages"] + [response],
            "iterations": state["iterations"] + 1,
        }
    
    return generate


def create_reflector(llm):
    """Create the content reflector/critic."""
    
    system_message = SystemMessage(
        content="""You are a thoughtful critic. Review the assistant's response and provide specific,
        constructive feedback on how it could be improved. Consider:
        - Accuracy and completeness
        - Clarity and structure
        - Tone and style appropriateness
        - Any missing important details
        
        If the response is excellent and needs no improvements, respond with exactly: "APPROVED"
        Otherwise, provide specific suggestions for improvement."""
    )
    
    def reflect(state: ReflectionState) -> dict:
        last_response = state["messages"][-1]
        
        critique_messages = [
            system_message,
            HumanMessage(content=f"Original request: {state['initial_query']}"),
            HumanMessage(content=f"Assistant's response: {last_response.content}"),
        ]
        
        response = llm.invoke(critique_messages)
        
        return {
            "messages": state["messages"] + [
                HumanMessage(content=f"Critique: {response.content}")
            ],
        }
    
    return reflect


print("Generator and reflector defined!")

Generator and reflector defined!


## Define Routing Logic

In [15]:
def should_continue(state: ReflectionState) -> str:
    """Decide whether to continue reflecting or end."""
    # Check if we've hit max iterations
    if state["iterations"] >= state["max_iterations"]:
        return "end"
    
    # Check if approved
    if state["messages"]:
        last_message = state["messages"][-1]
        if isinstance(last_message, HumanMessage) and "APPROVED" in last_message.content:
            return "end"
    
    # After generate (odd iterations), go to reflect
    # After reflect (even iterations), go back to generate
    if state["messages"]:
        last_message = state["messages"][-1]
        # If last message is from AI (generate node), go to reflect
        if isinstance(last_message, AIMessage):
            return "reflect"
        # If last message is critique (reflect node), go to generate
        elif isinstance(last_message, HumanMessage) and "Critique:" in last_message.content:
            return "generate"
    
    # Default: go to reflect
    return "reflect"

## Build the Reflection Graph

In [16]:
def create_reflection_graph(llm, max_iterations: int = 3):
    """Create the reflection agent graph."""
    
    generator = create_generator(llm)
    reflector = create_reflector(llm)
    
    workflow = StateGraph(ReflectionState)
    
    workflow.add_node("generate", generator)
    workflow.add_node("reflect", reflector)
    
    workflow.add_edge(START, "generate")
    
    workflow.add_conditional_edges(
        "generate",
        should_continue,
        {"reflect": "reflect", "end": END}
    )
    
    workflow.add_conditional_edges(
        "reflect",
        should_continue,
        {"generate": "generate", "end": END}
    )
    
    return workflow.compile()

## Initialize the Agent

In [17]:
# Initialize model
llm = ChatAmazonNova(
    model="nova-pro-v1",
    temperature=0.7,
)

# Create reflection agent
max_iterations = 4
agent = create_reflection_graph(llm, max_iterations)

print(f"Reflection agent initialized with max {max_iterations} iterations!")

Reflection agent initialized with max 4 iterations!


## Example 1: Short Essay

In [18]:
query = "Write a short essay on the importance of renewable energy"

result = agent.invoke({
    "messages": [HumanMessage(content=query)],
    "iterations": 0,
    "max_iterations": max_iterations,
    "initial_query": query,
})

# Get final output (last AI message)
final_output = None
for msg in reversed(result["messages"]):
    if isinstance(msg, AIMessage):
        final_output = msg.content
        break

print(f"Query: {query}\n")
print("=== Final Output ===")
print(final_output)
print(f"\n(Completed in {result['iterations']} iterations)")

Query: Write a short essay on the importance of renewable energy

=== Final Output ===
### The Importance of Renewable Energy

#### Introduction
The urgency of transitioning to renewable energy cannot be overstated. According to the Intergovernmental Panel on Climate Change (IPCC), limiting global warming to 1.5°C requires rapid and far-reaching transitions in energy systems. This essay aims to explore the multifaceted importance of renewable energy, outlining its environmental, economic, social, and political benefits. By examining specific data, examples, and addressing both opportunities and challenges, we will demonstrate how renewable energy sources—including solar, wind, hydro, geothermal, biomass, and tidal power—offer a sustainable alternative to traditional energy sources.

#### Environmental Benefits
Renewable energy plays a pivotal role in mitigating climate change and preserving biodiversity. 

**Reduction in Greenhouse Gases:**
- Renewable energy sources produce little to 

## Example 2: Explain a Concept

In [19]:
query = "Explain machine learning to a 12-year-old"

result = agent.invoke({
    "messages": [HumanMessage(content=query)],
    "iterations": 0,
    "max_iterations": max_iterations,
    "initial_query": query,
})

# Get final output
final_output = None
for msg in reversed(result["messages"]):
    if isinstance(msg, AIMessage):
        final_output = msg.content
        break

print(f"Query: {query}\n")
print("=== Final Output ===")
print(final_output)
print(f"\n(Completed in {result['iterations']} iterations)")

Query: Explain machine learning to a 12-year-old

=== Final Output ===
Great suggestions! Here’s the final revised version incorporating all your feedback:

---

### **What is Machine Learning?**

Imagine you have a pet robot dog. This robot dog doesn’t know how to do tricks yet. You want to teach it how to sit, roll over, and play dead. Machine learning is like teaching your robot dog new tricks. Instead of you writing specific instructions, the computer learns by looking at lots of examples and feedback.

### **How Does It Work?**

1. **Show It Examples**: You show the robot dog what "sit" looks like by pressing a button and making it sit. You do this many times.
2. **Give It Feedback**: Sometimes, the robot might not sit perfectly. You tell it, "Good job!" when it does it right and "Try again!" when it doesn’t.
3. **Learn from Mistakes**: Over time, the robot dog learns from its mistakes and gets better at sitting, rolling over, and playing dead.

### **Key Ideas**

- **Data**: This

## View Reflection Process

See the full generate-reflect-revise cycle.

In [20]:
print(f"Total messages: {len(result['messages'])}\n")
print("Reflection process:\n")

for i, msg in enumerate(result['messages']):
    if msg.type == "human":
        if "Critique:" in msg.content:
            print(f"{i+1}. [Critique] {msg.content[:100]}...")
        else:
            print(f"{i+1}. [User] {msg.content[:100]}...")
    elif msg.type == "ai":
        print(f"{i+1}. [Generated] {msg.content[:100]}...")
    print()

Total messages: 8

Reflection process:

1. [User] Explain machine learning to a 12-year-old...

2. [Generated] Sure! Let's imagine you have a pet robot dog. This robot dog doesn't know how to do tricks yet. You ...

3. [Critique] Critique: While the assistant's response is generally good, there are a few areas where it could be ...

4. [Generated] Great suggestions! Here’s a revised version of the explanation incorporating your feedback:

---

##...

5. [Critique] Critique: Here are some specific suggestions for improvement:

1. **Accuracy and Completeness**:
   ...

6. [Generated] Great suggestions! Here’s the final revised version incorporating all your feedback:

---

### **Wha...

7. [Critique] Critique: Here are some specific suggestions for improvement:

1. **Accuracy and Completeness**:
   ...

8. [Generated] Great suggestions! Here’s the final revised version incorporating all your feedback:

---

### **Wha...



## Try Your Own Query

In [21]:
# Modify the query below
your_query = "Write a professional email declining a meeting invitation politely"

result = agent.invoke({
    "messages": [HumanMessage(content=your_query)],
    "iterations": 0,
    "max_iterations": max_iterations,
    "initial_query": your_query,
})

# Get final output
final_output = None
for msg in reversed(result["messages"]):
    if isinstance(msg, AIMessage):
        final_output = msg.content
        break

print(f"Query: {your_query}\n")
print("=== Final Output ===")
print(final_output)
print(f"\n(Completed in {result['iterations']} iterations)")

Query: Write a professional email declining a meeting invitation politely

=== Final Output ===
Thank you for your thorough critique and suggestions. Here’s the final revised email incorporating all your feedback for accuracy, clarity, professionalism, and completeness:

---

**Subject: Regretfully Declining Meeting Invitation**

**Dear [Recipient's Name],**

I hope this email finds you well. Thank you very much for inviting me to the meeting regarding the Q3 Marketing Strategy scheduled for October 15th at 10:00 AM. I truly appreciate the opportunity and the consideration.

Unfortunately, I have a prior commitment that I am unable to reschedule—a mandatory team training session—and thus, I will not be able to attend the meeting. Please accept my sincerest apologies for any inconvenience this may cause.

The Q3 Marketing Strategy is very important to me, and I am keen to stay informed about the discussions and outcomes. If possible, could you please share the minutes or any key points 